This code is sourced from [this post](https://medium.com/@kunal.rustagi/boost-your-web-crawler-using-multiple-processes-in-python-3cc3ff519226).

이 노트북은 medium에서 Kunal Rustagi의 "Boost your web crwaler using multiple processes in Python"이란 포스트 내용을 한글로 번역하여 담고 있습니다. 자세한 내용은 원문을 참고하세요.

# Boost your web crawler using multiple processes in Python
# 멀티 프로세스를 사용해 파이썬 웹 크롤러 속도를 향상시키기.

이 글에서 우리는 멀티프로세싱을 사용해 크롤러가 더 빠르게 일하도록 만들겁니다.  


지금까지 우리는 웹페이지에서 데이터를 긁어 오는 것으로 데이터셋을 구축하고, 또 여러 웹페이지를 수집하려고 자체 웹 크롤러를 만들었습니다. 이 두가지 글의 링크는 [이곳](https://medium.com/@kunal.rustagi/web-scraping-for-beginners-using-python-73f0966d895f)과 [이곳](https://medium.com/@kunal.rustagi/building-a-web-crawler-to-scrape-data-from-multiple-pages-12799103b768)에서 찾을 수 있습니다.

---

## Timing, timing, timing!!
## 타이밍, 타이밍, 타이밍!!
### 멀티 프로세싱이 없이..

만약 이전에 작성한 웹 스크래핑 부분을 실행하면, 우리는 다음 방법으로 시간 주기를 계산할 수 있습니다.


In [2]:
import requests
from requests import get
from bs4 import BeautifulSoup
import numpy as np
import time
import random

pages = np.arange(1, 21, 1)

book_title = [] # 책 제목
star_rating = [] # 별점
product_price = [] # 가격

start = time.time()
for page in pages:
    time.sleep(random.randint(1, 3))
    
    url = 'http://books.toscrape.com/catalogue/page-' + str(page) + '.html'
    results = requests.get(url)
    soup = BeautifulSoup(results.text, 'html.parser')
    
    book_div = soup.find_all('li', class_='col-xs-6 col-sm-4 col-md-3 col-lg-3')
    
    for container in book_div:
        title = container.article.h3.a['title']
        book_title.append(title)
        
        price = container.article.find('div', class_ = 'product_price').p.text
        product_price.append(price)
        
        rating = container.article.p['class'][-1]
        star_rating.append(rating)

end = time.time()
print("It took {} seconds".format(end-start))

It took 55.3401141166687 seconds


이렇게 걸린 시간은 몹시 큽니다. 하지만 파이썬은 멀티 프로세싱을 제공합니다. 이 기능은 크롤링 시간을 상당히 줄일 수 있게 합니다.

## Multiprocessing
## 멀티프로세싱

파이썬의 [멀티프로세싱](https://docs.python.org/3.6/library/multiprocessing.html)은 스레딩 모듈과 유사한 API를 사용해 프로세스 생성을 지원하는 패키지입니다. 멀티프로세싱 패키지는 로컬과 원격의 동시성을 지원하고, 스레드 대신 subprocesses를 사용해 _Global Interpreter Lock_(전역 인터프리터 락)을 효과적으로 피할 수 있습니다. 이런 이유로 멀티프로세싱은 프로그래머가 컴퓨터의 멀티 프로세서를 완전히 효과적으로 사용할 수 있게 해줍니다. 멀티 프로세싱은 Unix와 Windows에서 모두 작동합니다.

쉽게 말해, 멀티 프로세싱은 컴퓨터의 프로세스를 여러개의 작은 프로세스로 쪼개고 이 프로세스들을 병렬적으로 실행하는 것입니다. (그래서 하드웨어에 크게 의존합니다.) 일반적으로 멀티프로세싱은 소프트웨어보다는 하드웨어적인 의미를 이야기합니다.

---

## 이전 코드를 수정해볼까요.
### 라이브러리, 스트럭처 그리고 페이지 인덱스

In [4]:
import requests
from bs4 import BeautifulSoup
import time
import random
import numpy as np
from multiprocessing import Pool # 멀티프로세싱

url_list = []

pages = np.arange(1, 51, 1)

이전 코드와 비교하면, 새로운 몇 줄이 추가되었습니다.
1. `multiprocessing` 라이브러리에서 `Pool` class를 import했습니다.
2. URL들을 저장할 `url_list` 리스트 변수를 만들었습니다.

## Creating new functions
## 새로운 함수 생성
이전 코드에서 몇몇 부분을 함수 안으로 옮길겁니다. 이 함수는 코드 반복을 피하게 하고 가독성을 올려주면서 복잡한 문제를 좀 더 단순한 문제로 만들어 줍니다.

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import numpy as np
from multiprocessing import Pool # 멀티프로세싱

url_list = []

pages = np.arange(1, 51, 1)

## 추가되는 부분
def generate_urls(pages):
    for page in pages:
        url = 'http://books.toscrape.com/catalogue/page-'+str(page)+'.html'
        url_list.append(url)

`generate_url()` 함수는 모든 urls을 생성하고 `url_list`에 추가합니다.

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import numpy as np
from multiprocessing import Pool # 멀티프로세싱

url_list = []

pages = np.arange(1, 31, 1)

def generate_urls():
    for page in pages:
        url = 'http://books.toscrape.com/catalogue/page-'+str(page)+'.html'
        url_list.append(url)
        
## 추가되는 부분
def scape_url(url):
    book_title = []
    star_rating = []
    product_price = []
    
    time.sleep(random.randint(1, 2))
    results = requests.get(url)
    soup = BeautifulSoup(results.text, 'html.parser')
    
    book_div = soup.find_all("li", class_ = "col-xs-6 col-sm-4 col-md-3 col-lg-3")
    
    for container in book_div:
        title = container.article.h3.a['title']
        book_title.append(title)
        
        price = container.article.find('div', class_='product_price').p.text
        product_price.append(price)
        
        rating = container.article.p['class'][-1]
        star_rating.append(rating)
        
    return (book_title, product_price, star_rating)

대부분의 웹 스크래핑 코드는 `scape_url()`로 옮겼습니다. 마지막에는 `book_title`, `product_price` 그리고 `star_rating` 리스트를 튜플로 감싸 반환합니다.

## Filling `url_list`
## `url_list` 채우기

이제 `generate_url()`함수를 호출하고 처음 10개의 urls을 출력해봅시다.

In [4]:
###### Pool() 부분에서 제대로 작동하지 않아서 원인 파악하기.
url_list = generate_urls()

In [6]:
url_list

In [5]:
for i in url_list[:4]:
    for (title, price, rating) in scape_url(i):
        print(title, price, rating)

TypeError: 'NoneType' object is not subscriptable

In [2]:
generate_urls()

for i in url_list[:10]:
    print(i)

http://books.toscrape.com/catalogue/page-1.html
http://books.toscrape.com/catalogue/page-2.html
http://books.toscrape.com/catalogue/page-3.html
http://books.toscrape.com/catalogue/page-4.html
http://books.toscrape.com/catalogue/page-5.html
http://books.toscrape.com/catalogue/page-6.html
http://books.toscrape.com/catalogue/page-7.html
http://books.toscrape.com/catalogue/page-8.html
http://books.toscrape.com/catalogue/page-9.html
http://books.toscrape.com/catalogue/page-10.html


## Pooling

In [3]:
from multiprocessing import cpu_count

cpu_count()

16

In [ ]:
p = Pool(10) # 10개의 워커를 생성함
book_list = p.map(scape_url, url_list) # 함수와 인자를 매핑하여 실행까지..?

In [ ]:
p.terminate()

In [ ]:
p.join()

+ `Pool()` 클래스는 한번에 얼만큼의 워크 프로세스를 생성할지를 말합니다.여기서는 10을 인수로 전달했는데, 10개의 URL들이 동시에 처리하게 됩니다.
+ `map()`은 첫번째 인자로 `scrape_url`을 그리고 두번째 인자로 `url_list`를 받습니다. 50개의 페이지들은 한번에 10개씩 처리됩니다. 그러면 5번의 반복이 수행될 겁니다.
+ `terminate()`는 프로세스를 제거합니다.
+ `join()`은 워크 프로세스가 빠져나올때까지 기다리기 위해 사용됩니다.

---

## Check time taken
## 걸린 시간 확인하기
`start`와 `end`를 코드에 추가해서 시간을 계산해보겠습니다.